In [ ]:
print("hi there")

In [2]:
from itertools import combinations
import numpy as np
from matplotlib import pyplot as plt

from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
from transformers.models.llama.modeling_llama import LlamaForCausalLM
from transformers import TemperatureLogitsWarper, LogitsProcessorList
import pandas as pd
from random import randint
import gc

In [3]:
# tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
# model = AutoModelForCausalLM.from_pretrained("huggyllama/llama-7b")
tokenizer = LlamaTokenizer.from_pretrained("lmsys/vicuna-13b-v1.3")
model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-13b-v1.3")

# tokenizer = LlamaTokenizer.from_pretrained("lmsys/vicuna-7b-v1.3")
# model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-7b-v1.3")

model.cuda()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [4]:
def recover_oom():
    gc.collect()
    torch.cuda.empty_cache()

recover_oom()


def infer(func, prompt, repeats=5, verbose=False):
    sep = "\n***********\n\n"
    if verbose:
        print(prompt, end=sep)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    input_ids = input_ids.repeat((repeats, 1))
    gen_output = func(input_ids)
    responses = [tokenizer.decode(sequence).replace('<unk>', '') for sequence in gen_output]
    responses = [r[len(prompt) + 5:].strip() for r in responses]
    if verbose:
        print(*responses, sep=sep)
    return responses


In [5]:
# def sampling(tokens):
#     return model.sample(
#         tokens, logits_warper=logits_warper, max_length=80, eos_token_id=[2, 13]
#     )

def sampling(tokens):
    return model.generate(tokens, max_new_tokens=50, eos_token_id=[2, 13], do_sample=True)


In [6]:
# open the feud dataset
import json

with open('../data_store/question_db.json', 'r') as f:
    questions = json.load(f)


print(len(questions))
lengths = [len(q['answers']) for q in questions]
from collections import Counter
print(Counter(lengths))

13108
Counter({6: 5019, 5: 3867, 4: 2935, 3: 1216, 2: 71})


In [7]:
qs = [q for q in questions if len(q['answers']) == 3][:5]
questions = [q for q in questions if len(q['answers']) == 6]
# print(qa[0])
# prompt = qa[0]['question']

In [8]:
def template(prompt):
    instruction = 'Your task is to play Family Feud. Provide one brief answer you believe the majority of people would agree with.\n'
    instruction += "\n\n".join([
        f"Question: {q['question']}\nAnswer: {q['answers'][2]['text']}" for q in qs[:5]
    ])
    instruction += f"\n\nQuestion: {prompt}\nAnswer: "
    return instruction

In [9]:
i = 1
question = questions[i]
print(question)
responses = infer(sampling, template(question['question']), repeats=6)
print(responses)

{'question': 'Name Something Your Spouse Might Get Mad At You for Doing Too Much', 'link': '/question/name-something-your-spouse-might-get-mad-at-you-for-doing-too-much', 'answers': [{'text': 'Talking', 'points': 27}, {'text': 'Shopping/Spending', 'points': 21}, {'text': 'Drinking', 'points': 18}, {'text': 'Sleeping', 'points': 16}, {'text': 'Spending Money', 'points': 9}, {'text': 'Watching TV', 'points': 6}]}


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


['0 hours of TV', '', 'ing Too Frequently', '', 'BBQing', '']


In [10]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer("all-mpnet-base-v2")


In [11]:
print(responses)
print([a['text'] for a in question['answers']])

['0 hours of TV', '', 'ing Too Frequently', '', 'BBQing', '']
['Talking', 'Shopping/Spending', 'Drinking', 'Sleeping', 'Spending Money', 'Watching TV']


In [12]:
# for i in range(5):
#     question = questions[i]
#     responses = infer(sampling, template(question['question']), repeats=6)
#     similarities = util.cos_sim(sbert.encode(responses),  sbert.encode([a['text'] for a in question['answers']]))
#     # plt.imshow(similarities.numpy(), cmap='plasma', vmin=0, vmax=1) #RdYlGn' gist_rainbow
#     # plt.colorbar()
#     # plt.title(i)
#     # plt.show()
#     for (id1, id2) in (similarities > 0.7).nonzero():
#         print('--', similarities[id1, id2])
#         print(question['question'])
#         print(responses[id1])
#         print(question['answers'][id2]['text'])



In [14]:
SIMILARITY_THRESHOLD = 0.75
def count_cover(responses, answers, similarity_threshold=SIMILARITY_THRESHOLD, encoder=sbert):
    """
    Returns the share of answers that was guessed by responses
    The similarity between answers and responses measured by cosine sim on encoder embeddings
    Cut off by similarity threshold
    Note, each response could "cover" only one answer with top similarity
    Answer is considered guessed, if there are at least one reponse with similarity above threshold
    """
    similarities = util.cos_sim(encoder.encode(responses), encoder.encode(answers))
    max_sims = similarities * (similarities.max(dim=1).values[:, None] == similarities).float()
    coverage = ((max_sims > similarity_threshold).float().mean(dim=0) > 0).float().mean()
    return coverage


In [ ]:
RESTRICT = 20
NUM_RUNS = 5
def sampling(tokens):
    return model.generate(tokens, max_new_tokens=50, eos_token_id=[2, 13], do_sample=True)

averages = []
for _ in range(NUM_RUNS):
    coverages = []
    for question in tqdm(questions[:RESTRICT]):
        answers = [a['text'] for a in question['answers']]
        responses = infer(sampling, template(question['question']), repeats=20)
        coverages.append(count_cover(responses, answers))

        print(question['question'])
        print(answers)
        print(responses)


    print([c.item() for c in coverages])
    print(np.mean(coverages))
    averages.append(np.mean(coverages))

  0%|          | 0/20 [00:00<?, ?it/s]

Which Former President Would Look Funny Wearing A Dress?
['George W Bush', 'William Clinton', 'Abe Lincoln', 'Richard Nixon', 'George Washington', 'Ronald Reagan']
['George W. Bush</s>', '2 George W. Bush', '2nd President Bill Clinton', '0. Grover Cleveland', '3</s>', '2 (George H.W. Bush)', '2 Ronald Reagan', '2', '', '4. Ulysses S. Grant', 'Benjamin Harrison', '', '2nd President of the United States Bill Clinton', '2</s>', 'President George Washington</s>', '4', '5', '', 'nd George W. Bush / 44th Barack Obama or 45th Donald Trump', '3</s>']
Name Something Your Spouse Might Get Mad At You for Doing Too Much
['Talking', 'Shopping/Spending', 'Drinking', 'Sleeping', 'Spending Money', 'Watching TV']
['난', 'Working too much', '', '', '不得回答）', '', '', '', '', '', '.', 'Betting On Sports Games', '不在规范。', '-Play Xbox Games', '', '. Over Spending', '', '-Year-Old’s Birthday Party</s>', '', '. Clean the whole house']
What Kind Of Gift Would You Get Somebody That Spends A Lot Of Time At The Beac

In [ ]:
averages